In [71]:
from random import random, choice, randint
from collections import namedtuple
from pprint import pprint, pformat
import random
import numpy as np
from copy import copy, deepcopy
import logging
from dataclasses import dataclass


In [72]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [73]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self): #The __bool__ method returns True if there are objects remaining in any row, indicating that the game is not over
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


In [74]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [75]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [77]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


In [78]:
POPULATION_SIZE = 30
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .80
RULES_SIZE=3
size_game=4
list_strategies= [pure_random, gabriele, optimal]


In [79]:

@dataclass
class Individual():
    genotype: []
    fitness : float


def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]        #list of possibleMove elements
    champion = max(pool, key=lambda individual: individual.fitness)     #one possibleMove element with the best nim_sum
    return champion


def mutate(individual) :     #mutates the number of objects to remove from the same unchanged row OR mutates the row 
    genotype= individual.genotype
    # Choose an index to change
    index_to_change = random.randint(0, len(genotype) - 1)

    # Generate a random value for the change
    change = random.uniform(-0.1, 0.1)  # You can adjust the range as needed

    # Apply the change to the chosen index
    genotype[index_to_change] += change

    # Ensure the modified value is between 0 and 1
    genotype[index_to_change] = max(0, min(1, genotype[index_to_change]))

    # Adjust the other values to maintain the sum equal to 1
    total = sum(genotype)
    genotype = [value / total for value in genotype]

    individual.genotype= genotype
    return individual



In [80]:
def play(first,genotype , strategy_player1):
    # Choose the best PossibleMove in the population and play Nim against the pure random strategy
    
    nim = Nim(size_game)
    player = first
    while nim:
        chosen_strat = random.choices(list_strategies, genotype)[0]
        strategy = ( chosen_strat, strategy_player1)
        ply = strategy[player](nim)
        nim.nimming(ply)
        player = 1 - player
    
    return player


In [81]:
def fitness(genotype):#returns the rate of winning the game after playing it 'nb_game' times with one same genotype mbut against differents types of opponent's strategies
    win=0
    first=0
    strategies_opponent= list_strategies
    nb_game=20
    for i in range (nb_game):
        opponent= strategies_opponent[i%RULES_SIZE] # A verifier les effets de bords
        val= play(first, genotype, opponent)
        first=1-first
        if val == 0:
            win+=1
    
    win_rate= win/nb_game
    return win_rate

In [82]:
def random_genotype():      #returns a random genotype
    random_genotype=[]
    for i in range(RULES_SIZE):
        random_weights = [random.random() for _ in range(RULES_SIZE)]
        
    total = sum(random_weights)
    random_genotype = [value / total for value in random_weights]
    
    return random_genotype

In [83]:
def evolved_startegy():
    population = [
        Individual(
            fitness=None,
            genotype = random_genotype()              
        )
        for _ in range(POPULATION_SIZE)
    ]

    for i in population:
        i.fitness = fitness(i.genotype)  # Use the fitness function on the nim_move

    for generation in range(10):
        offspring = list()
        for counter in range(OFFSPRING_SIZE):
            p1 = select_parent(population)
            if random.random() < MUTATION_PROBABILITY:                                  
                p1 = mutate(p1)
                            
            offspring.append(p1)

        for i in offspring:
            i.fitness = fitness(i.genotype) 
            
        population.extend(offspring)
        population.sort(key=lambda i: i.fitness, reverse = True)           
        population = population[:POPULATION_SIZE]

    # Choose the best individual in the population and return its genotype
    print('best ind')
    print(population[0].fitness)
    print(population[0].genotype)
    best_individual = population[0]
    return best_individual.genotype

## Oversimplified match

In [84]:


chosen_genotype=evolved_startegy()
first=1
player=play(first,chosen_genotype, pure_random)
print(' The player who won is')
print(player)



best ind
0.6
[0.44592888898377847, 0.08738387979706358, 0.46668723121915784]
 The player who won is
1
